<a href="https://colab.research.google.com/github/Arun-Alpy/PySpark/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linux Basic

In [15]:
!git version

git version 2.34.1


In [ ]:
print ("PySpark")

PySpark


In [16]:
!whoami

root


In [17]:
!pwd

/content


# PySpark Basics

In [18]:
!pip install pyspark

In [19]:
!pip show pyspark

Name: pyspark
Version: 3.5.1
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: py4j
Required-by: dataproc-spark-connect


In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('Basics').getOrCreate()

In [25]:
#create dataframe
data=[("Hello","World")]
columns=["Word1","Word2"]
df=spark.createDataFrame(data,columns)

In [27]:
df.show()

+-----+-----+
|Word1|Word2|
+-----+-----+
|Hello|World|
+-----+-----+



## Basic Transformations and Actions

In [3]:
columns=["Name","Department","Salary"]
data = [
    ("John", "Sales", 3000),
    ("Jane", "Finance", 4000),
    ("Mike", "Sales", 3500),
    ("Alice", "Finance", 3800),
    ("Bob", "IT", 4500)
]
df = spark.createDataFrame(data, columns)
df.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
| John|     Sales|  3000|
| Jane|   Finance|  4000|
| Mike|     Sales|  3500|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [29]:
#filter:employee with salary >3500
df_filtered=df.filter(df.Salary>3500)
df_filtered.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
| Jane|   Finance|  4000|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [31]:
#group by and aggregate: Avg salary by Department
df_grouped=df.groupBy("Department").agg({"Salary":"avg"})
df_grouped.show()

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|     Sales|     3250.0|
|   Finance|     3900.0|
|        IT|     4500.0|
+----------+-----------+



In [32]:
#group by and aggregate: Avg salary by Department
df_grouped=df.groupBy("Department").avg("Salary")
df_grouped.show()

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|     Sales|     3250.0|
|   Finance|     3900.0|
|        IT|     4500.0|
+----------+-----------+



In [33]:
#Add a new column: Salary with bonus (10%)
from pyspark.sql.functions import col
exp=col("Salary")*1.1
df_with_bonus=df.withColumn("Salary_10%_bonus",exp)
df_with_bonus.show()

+-----+----------+------+------------------+
| Name|Department|Salary|  Salary_10%_bonus|
+-----+----------+------+------------------+
| John|     Sales|  3000|3300.0000000000005|
| Jane|   Finance|  4000|            4400.0|
| Mike|     Sales|  3500|3850.0000000000005|
|Alice|   Finance|  3800|            4180.0|
|  Bob|        IT|  4500|            4950.0|
+-----+----------+------+------------------+



In [34]:

from pyspark.sql.functions import col,upper,lower,concat_ws,length,when

df.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
| John|     Sales|  3000|
| Jane|   Finance|  4000|
| Mike|     Sales|  3500|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [35]:
#Change case transformation
df_upper=df.withColumn("Name_upper",upper(col("Name")))
df_lower=df.withColumn("Name_lower",lower(col("Name")))
df_upper.show()
df_lower.show()

+-----+----------+------+----------+
| Name|Department|Salary|Name_upper|
+-----+----------+------+----------+
| John|     Sales|  3000|      JOHN|
| Jane|   Finance|  4000|      JANE|
| Mike|     Sales|  3500|      MIKE|
|Alice|   Finance|  3800|     ALICE|
|  Bob|        IT|  4500|       BOB|
+-----+----------+------+----------+

+-----+----------+------+----------+
| Name|Department|Salary|Name_lower|
+-----+----------+------+----------+
| John|     Sales|  3000|      john|
| Jane|   Finance|  4000|      jane|
| Mike|     Sales|  3500|      mike|
|Alice|   Finance|  3800|     alice|
|  Bob|        IT|  4500|       bob|
+-----+----------+------+----------+



In [36]:
#concatenate columns
df_concat=df.withColumn("Full_Info",concat_ws(",","Name","Department"))
df_concat.show()

+-----+----------+------+-------------+
| Name|Department|Salary|    Full_Info|
+-----+----------+------+-------------+
| John|     Sales|  3000|   John,Sales|
| Jane|   Finance|  4000| Jane,Finance|
| Mike|     Sales|  3500|   Mike,Sales|
|Alice|   Finance|  3800|Alice,Finance|
|  Bob|        IT|  4500|       Bob,IT|
+-----+----------+------+-------------+



In [37]:
# string length of names in new DF
df_length=df.withColumn("Name_length",length(col("Name")))
df_length.show()

+-----+----------+------+-----------+
| Name|Department|Salary|Name_length|
+-----+----------+------+-----------+
| John|     Sales|  3000|          4|
| Jane|   Finance|  4000|          4|
| Mike|     Sales|  3500|          4|
|Alice|   Finance|  3800|          5|
|  Bob|        IT|  4500|          3|
+-----+----------+------+-----------+



In [38]:
#conditional Columns(Salary category)
df_conditional=df.withColumn("Salary_Category",
                              when(col("Salary")>3500,"High")
                              .when(col("Salary")>3000,"Medium")
                              .otherwise("Low"))
df_conditional.show()

+-----+----------+------+---------------+
| Name|Department|Salary|Salary_Category|
+-----+----------+------+---------------+
| John|     Sales|  3000|            Low|
| Jane|   Finance|  4000|           High|
| Mike|     Sales|  3500|         Medium|
|Alice|   Finance|  3800|           High|
|  Bob|        IT|  4500|           High|
+-----+----------+------+---------------+



In [39]:
# rename column (Salary to Base_Salary)
df_renamed=df.withColumnRenamed("Salary","Base_Salary")
df_renamed.show()

+-----+----------+-----------+
| Name|Department|Base_Salary|
+-----+----------+-----------+
| John|     Sales|       3000|
| Jane|   Finance|       4000|
| Mike|     Sales|       3500|
|Alice|   Finance|       3800|
|  Bob|        IT|       4500|
+-----+----------+-----------+



In [4]:
#group by department and count employees

df.groupBy("Department").count().show()

+----------+-----+
|Department|count|
+----------+-----+
|     Sales|    2|
|   Finance|    2|
|        IT|    1|
+----------+-----+



In [5]:
df.groupBy("Department").agg({"Salary":"sum"}).show()

+----------+-----------+
|Department|sum(Salary)|
+----------+-----------+
|     Sales|       6500|
|   Finance|       7800|
|        IT|       4500|
+----------+-----------+



In [7]:
#group by department and calculate multiple Aggregations
from pyspark.sql.functions import avg, max, min
df.groupBy("Department").agg(
    avg("Salary").alias("avg_salary"),
    max("Salary").alias("max_salary"),
    min("Salary").alias("min_salary")
).show()

+----------+----------+----------+----------+
|Department|avg_salary|max_salary|min_salary|
+----------+----------+----------+----------+
|     Sales|    3250.0|      3500|      3000|
|   Finance|    3900.0|      4000|      3800|
|        IT|    4500.0|      4500|      4500|
+----------+----------+----------+----------+



In [8]:
# Create another DataFrame for department info
dept_data = [
    ("Sales", "Building A"),
    ("Finance", "Building B"),
    ("IT", "Building C")
]
dept_columns = ["Department", "Location"]

In [9]:
df.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
| John|     Sales|  3000|
| Jane|   Finance|  4000|
| Mike|     Sales|  3500|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [10]:
dept_df=spark.createDataFrame(dept_data,dept_columns)
dept_df.show()


+----------+----------+
|Department|  Location|
+----------+----------+
|     Sales|Building A|
|   Finance|Building B|
|        IT|Building C|
+----------+----------+



In [11]:
#join employees with department info

joined_df=df.join(dept_df,on="Department",how="inner")
joined_df.show()

+----------+-----+------+----------+
|Department| Name|Salary|  Location|
+----------+-----+------+----------+
|   Finance| Jane|  4000|Building B|
|   Finance|Alice|  3800|Building B|
|        IT|  Bob|  4500|Building C|
|     Sales| John|  3000|Building A|
|     Sales| Mike|  3500|Building A|
+----------+-----+------+----------+



In [13]:
# Employee DataFrame
emp_data = [
    (1, "John", "Sales", 3000),
    (2, "Jane", "Finance", 4000),
    (3, "Mike", "Sales", 3500),
    (4, "Alice", "HR", 3800),
    (5, "Bob", "IT", 4500),
    (6, "Sam", "Support", 3200)
]
emp_cols = ["EmpID", "Name", "Department", "Salary"]
emp_df = spark.createDataFrame(emp_data, emp_cols)

# Department DataFrame
dept_data = [
    ("Sales", "Building A"),
    ("Finance", "Building B"),
    ("IT", "Building C"),
    ("Admin", "Building D")
]
dept_cols = ["Department", "Location"]
dept_df = spark.createDataFrame(dept_data, dept_cols)

# Display both
emp_df.show()
dept_df.show()

+-----+-----+----------+------+
|EmpID| Name|Department|Salary|
+-----+-----+----------+------+
|    1| John|     Sales|  3000|
|    2| Jane|   Finance|  4000|
|    3| Mike|     Sales|  3500|
|    4|Alice|        HR|  3800|
|    5|  Bob|        IT|  4500|
|    6|  Sam|   Support|  3200|
+-----+-----+----------+------+

+----------+----------+
|Department|  Location|
+----------+----------+
|     Sales|Building A|
|   Finance|Building B|
|        IT|Building C|
|     Admin|Building D|
+----------+----------+



In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, month


In [15]:
spark = SparkSession.builder.appName("SalesAnalysis").getOrCreate()



In [16]:
sales_data = [
    (1001, 101, 501, "2025-07-10 08:23:00", 3, 25.5),
    (1002, 102, 502, "2025-07-11 09:45:00", 2, 15.0),
    (1003, 103, 503, "2025-07-12 10:15:00", 1, 30.0),
    (1004, 101, 504, "2025-07-13 12:20:00", 5, 25.5),
    (1005, 105, 505, "2025-07-14 14:35:00", 10, 45.0),
    (1006, 102, 506, "2025-07-15 16:00:00", 4, 15.0)
]
sales_columns = ["sale_id", "product_id", "customer_id", "sale_date", "quantity", "price"]
sales_df = spark.createDataFrame(sales_data, sales_columns)

In [17]:
# Product Dataset
product_data = [
    (101, "Widget A", "Gadgets"),
    (102, "Widget B", "Gadgets"),
    (103, "Widget C", "Electronics"),
    (104, "Widget D", "Electronics"),
    (105, "Widget E", "Home & Living")
]
product_columns = ["product_id", "product_name", "category"]
product_df = spark.createDataFrame(product_data, product_columns)


In [18]:
customer_data = [
    (501, "Alice", "alice@example.com", "2025-05-20 10:10:00"),
    (502, "Bob", "bob@example.com", "2025-06-15 14:00:00"),
    (503, "Charlie", "charlie@example.com", "2025-04-05 09:50:00"),
    (504, "David", "david@example.com", "2025-07-01 12:25:00"),
    (505, "Emma", "emma@example.com", "2025-07-10 15:30:00"),
    (506, "Frank", "frank@example.com", "2025-03-23 17:00:00")
]
customer_columns = ["customer_id", "customer_name", "email", "join_date"]
customer_df = spark.createDataFrame(customer_data, customer_columns)


In [19]:
total_revenue_per_product = sales_df.withColumn("revenue", col("quantity") * col("price")) \
    .groupBy("product_id") \
    .agg(sum("revenue").alias("total_revenue"))

total_revenue_per_product.show()

+----------+-------------+
|product_id|total_revenue|
+----------+-------------+
|       103|         30.0|
|       101|        204.0|
|       102|         90.0|
|       105|        450.0|
+----------+-------------+



In [20]:
total_quantity_per_customer = sales_df.groupBy("customer_id") \
    .agg(sum("quantity").alias("total_quantity"))

total_quantity_per_customer.show()



+-----------+--------------+
|customer_id|total_quantity|
+-----------+--------------+
|        502|             2|
|        501|             3|
|        503|             1|
|        504|             5|
|        506|             4|
|        505|            10|
+-----------+--------------+



In [21]:
average_revenue_per_customer = sales_df.withColumn("revenue", col("quantity") * col("price")) \
    .groupBy("customer_id") \
    .agg(avg("revenue").alias("average_revenue"))

average_revenue_per_customer.show()

+-----------+---------------+
|customer_id|average_revenue|
+-----------+---------------+
|        502|           30.0|
|        501|           76.5|
|        503|           30.0|
|        504|          127.5|
|        506|           60.0|
|        505|          450.0|
+-----------+---------------+



In [22]:
monthly_sales_total = sales_df.withColumn("month", month(col("sale_date"))) \
    .withColumn("revenue", col("quantity") * col("price")) \
    .groupBy("month") \
    .agg(sum("revenue").alias("total_monthly_revenue"))

monthly_sales_total.show()

+-----+---------------------+
|month|total_monthly_revenue|
+-----+---------------------+
|    7|                774.0|
+-----+---------------------+



In [23]:
sales_with_category = sales_df.join(product_df, "product_id")
sales_per_category = sales_with_category.groupBy("category") \
    .count() \
    .withColumnRenamed("count", "total_sales")

sales_per_category.show()

+-------------+-----------+
|     category|total_sales|
+-------------+-----------+
|  Electronics|          1|
|      Gadgets|          4|
|Home & Living|          1|
+-------------+-----------+



In [24]:
from pyspark.sql.functions import desc

top_3_products = sales_df.withColumn("revenue", col("quantity") * col("price")) \
    .groupBy("product_id") \
    .agg(sum("revenue").alias("total_revenue")) \
    .orderBy(desc("total_revenue")) \
    .limit(3)

top_3_products.show()

+----------+-------------+
|product_id|total_revenue|
+----------+-------------+
|       105|        450.0|
|       101|        204.0|
|       102|         90.0|
+----------+-------------+



In [25]:
sales_with_product_info = sales_df.join(product_df, "product_id")
sales_with_product_info.show()

+----------+-------+-----------+-------------------+--------+-----+------------+-------------+
|product_id|sale_id|customer_id|          sale_date|quantity|price|product_name|     category|
+----------+-------+-----------+-------------------+--------+-----+------------+-------------+
|       101|   1001|        501|2025-07-10 08:23:00|       3| 25.5|    Widget A|      Gadgets|
|       101|   1004|        504|2025-07-13 12:20:00|       5| 25.5|    Widget A|      Gadgets|
|       102|   1002|        502|2025-07-11 09:45:00|       2| 15.0|    Widget B|      Gadgets|
|       102|   1006|        506|2025-07-15 16:00:00|       4| 15.0|    Widget B|      Gadgets|
|       103|   1003|        503|2025-07-12 10:15:00|       1| 30.0|    Widget C|  Electronics|
|       105|   1005|        505|2025-07-14 14:35:00|      10| 45.0|    Widget E|Home & Living|
+----------+-------+-----------+-------------------+--------+-----+------------+-------------+



In [26]:
sales_with_customer_info = sales_df.join(customer_df, "customer_id")
sales_with_customer_info.show()

+-----------+-------+----------+-------------------+--------+-----+-------------+-------------------+-------------------+
|customer_id|sale_id|product_id|          sale_date|quantity|price|customer_name|              email|          join_date|
+-----------+-------+----------+-------------------+--------+-----+-------------+-------------------+-------------------+
|        501|   1001|       101|2025-07-10 08:23:00|       3| 25.5|        Alice|  alice@example.com|2025-05-20 10:10:00|
|        502|   1002|       102|2025-07-11 09:45:00|       2| 15.0|          Bob|    bob@example.com|2025-06-15 14:00:00|
|        503|   1003|       103|2025-07-12 10:15:00|       1| 30.0|      Charlie|charlie@example.com|2025-04-05 09:50:00|
|        504|   1004|       101|2025-07-13 12:20:00|       5| 25.5|        David|  david@example.com|2025-07-01 12:25:00|
|        505|   1005|       105|2025-07-14 14:35:00|      10| 45.0|         Emma|   emma@example.com|2025-07-10 15:30:00|
|        506|   1006|   

In [27]:
inner_join_gadgets = sales_df.join(product_df, "product_id", "inner") \
    .filter(product_df.category == "Gadgets")

inner_join_gadgets.show()

+----------+-------+-----------+-------------------+--------+-----+------------+--------+
|product_id|sale_id|customer_id|          sale_date|quantity|price|product_name|category|
+----------+-------+-----------+-------------------+--------+-----+------------+--------+
|       101|   1001|        501|2025-07-10 08:23:00|       3| 25.5|    Widget A| Gadgets|
|       101|   1004|        504|2025-07-13 12:20:00|       5| 25.5|    Widget A| Gadgets|
|       102|   1002|        502|2025-07-11 09:45:00|       2| 15.0|    Widget B| Gadgets|
|       102|   1006|        506|2025-07-15 16:00:00|       4| 15.0|    Widget B| Gadgets|
+----------+-------+-----------+-------------------+--------+-----+------------+--------+



In [28]:
left_join_sales_product = sales_df.join(product_df, "product_id", "left")
left_join_sales_product.show()

+----------+-------+-----------+-------------------+--------+-----+------------+-------------+
|product_id|sale_id|customer_id|          sale_date|quantity|price|product_name|     category|
+----------+-------+-----------+-------------------+--------+-----+------------+-------------+
|       103|   1003|        503|2025-07-12 10:15:00|       1| 30.0|    Widget C|  Electronics|
|       101|   1001|        501|2025-07-10 08:23:00|       3| 25.5|    Widget A|      Gadgets|
|       102|   1002|        502|2025-07-11 09:45:00|       2| 15.0|    Widget B|      Gadgets|
|       105|   1005|        505|2025-07-14 14:35:00|      10| 45.0|    Widget E|Home & Living|
|       101|   1004|        504|2025-07-13 12:20:00|       5| 25.5|    Widget A|      Gadgets|
|       102|   1006|        506|2025-07-15 16:00:00|       4| 15.0|    Widget B|      Gadgets|
+----------+-------+-----------+-------------------+--------+-----+------------+-------------+



In [29]:
from pyspark.sql.functions import col

sales_alias1 = sales_df.alias("s1")
sales_alias2 = sales_df.alias("s2")

self_join_df = sales_alias1.join(sales_alias2, col("s1.product_id") == col("s2.product_id")) \
    .filter(col("s1.sale_id") < col("s2.sale_id"))

self_join_df.select("s1.product_id", "s1.sale_date", "s2.sale_date").show()

+----------+-------------------+-------------------+
|product_id|          sale_date|          sale_date|
+----------+-------------------+-------------------+
|       101|2025-07-10 08:23:00|2025-07-13 12:20:00|
|       102|2025-07-11 09:45:00|2025-07-15 16:00:00|
+----------+-------------------+-------------------+



In [30]:
full_outer_join_sales_product = sales_df.join(product_df, "product_id", "full")
full_outer_join_sales_product.show()


+----------+-------+-----------+-------------------+--------+-----+------------+-------------+
|product_id|sale_id|customer_id|          sale_date|quantity|price|product_name|     category|
+----------+-------+-----------+-------------------+--------+-----+------------+-------------+
|       101|   1001|        501|2025-07-10 08:23:00|       3| 25.5|    Widget A|      Gadgets|
|       101|   1004|        504|2025-07-13 12:20:00|       5| 25.5|    Widget A|      Gadgets|
|       102|   1002|        502|2025-07-11 09:45:00|       2| 15.0|    Widget B|      Gadgets|
|       102|   1006|        506|2025-07-15 16:00:00|       4| 15.0|    Widget B|      Gadgets|
|       103|   1003|        503|2025-07-12 10:15:00|       1| 30.0|    Widget C|  Electronics|
|       104|   NULL|       NULL|               NULL|    NULL| NULL|    Widget D|  Electronics|
|       105|   1005|        505|2025-07-14 14:35:00|      10| 45.0|    Widget E|Home & Living|
+----------+-------+-----------+------------------

In [31]:
complete_df = sales_df.join(product_df, "product_id") \
    .join(customer_df, "customer_id")

complete_df.show()

+-----------+----------+-------+-------------------+--------+-----+------------+-------------+-------------+-------------------+-------------------+
|customer_id|product_id|sale_id|          sale_date|quantity|price|product_name|     category|customer_name|              email|          join_date|
+-----------+----------+-------+-------------------+--------+-----+------------+-------------+-------------+-------------------+-------------------+
|        502|       102|   1002|2025-07-11 09:45:00|       2| 15.0|    Widget B|      Gadgets|          Bob|    bob@example.com|2025-06-15 14:00:00|
|        501|       101|   1001|2025-07-10 08:23:00|       3| 25.5|    Widget A|      Gadgets|        Alice|  alice@example.com|2025-05-20 10:10:00|
|        503|       103|   1003|2025-07-12 10:15:00|       1| 30.0|    Widget C|  Electronics|      Charlie|charlie@example.com|2025-04-05 09:50:00|
|        504|       101|   1004|2025-07-13 12:20:00|       5| 25.5|    Widget A|      Gadgets|        Davi

In [32]:
from pyspark.sql.functions import col

filtered_sales_df = sales_df.filter(col("sale_date").between("2025-07-10", "2025-07-15"))
filtered_sales_df.show()

+-------+----------+-----------+-------------------+--------+-----+
|sale_id|product_id|customer_id|          sale_date|quantity|price|
+-------+----------+-----------+-------------------+--------+-----+
|   1001|       101|        501|2025-07-10 08:23:00|       3| 25.5|
|   1002|       102|        502|2025-07-11 09:45:00|       2| 15.0|
|   1003|       103|        503|2025-07-12 10:15:00|       1| 30.0|
|   1004|       101|        504|2025-07-13 12:20:00|       5| 25.5|
|   1005|       105|        505|2025-07-14 14:35:00|      10| 45.0|
+-------+----------+-----------+-------------------+--------+-----+



In [33]:
from pyspark.sql.functions import desc

top_5_customers = sales_df.withColumn("total_spend", col("quantity") * col("price")) \
    .join(customer_df, "customer_id") \
    .groupBy("customer_id", "customer_name") \
    .agg(sum("total_spend").alias("total_spending")) \
    .orderBy(desc("total_spending")) \
    .limit(5)

top_5_customers.show()

+-----------+-------------+--------------+
|customer_id|customer_name|total_spending|
+-----------+-------------+--------------+
|        505|         Emma|         450.0|
|        504|        David|         127.5|
|        501|        Alice|          76.5|
|        506|        Frank|          60.0|
|        502|          Bob|          30.0|
+-----------+-------------+--------------+



In [34]:
from pyspark.sql.functions import when

customer_spending = sales_df.withColumn("total_spend", col("quantity") * col("price")) \
    .join(customer_df, "customer_id") \
    .groupBy("customer_id", "customer_name") \
    .agg(sum("total_spend").alias("total_spending"))

categorized_customers = customer_spending.withColumn("spending_category",
    when(col("total_spending") < 100, "Low")
    .when((col("total_spending") >= 100) & (col("total_spending") <= 300), "Medium")
    .otherwise("High")
)

categorized_customers.show()

+-----------+-------------+--------------+-----------------+
|customer_id|customer_name|total_spending|spending_category|
+-----------+-------------+--------------+-----------------+
|        501|        Alice|          76.5|              Low|
|        502|          Bob|          30.0|              Low|
|        503|      Charlie|          30.0|              Low|
|        504|        David|         127.5|           Medium|
|        505|         Emma|         450.0|             High|
|        506|        Frank|          60.0|              Low|
+-----------+-------------+--------------+-----------------+



In [35]:
from pyspark.sql.functions import min, max

customer_purchase_dates = sales_df.groupBy("customer_id") \
    .agg(
        min("sale_date").alias("first_purchase_date"),
        max("sale_date").alias("last_purchase_date")
    )

customer_purchase_dates_with_names = customer_purchase_dates.join(customer_df, "customer_id") \
    .select("customer_id", "customer_name", "first_purchase_date", "last_purchase_date")

customer_purchase_dates_with_names.show()

+-----------+-------------+-------------------+-------------------+
|customer_id|customer_name|first_purchase_date| last_purchase_date|
+-----------+-------------+-------------------+-------------------+
|        502|          Bob|2025-07-11 09:45:00|2025-07-11 09:45:00|
|        501|        Alice|2025-07-10 08:23:00|2025-07-10 08:23:00|
|        503|      Charlie|2025-07-12 10:15:00|2025-07-12 10:15:00|
|        504|        David|2025-07-13 12:20:00|2025-07-13 12:20:00|
|        506|        Frank|2025-07-15 16:00:00|2025-07-15 16:00:00|
|        505|         Emma|2025-07-14 14:35:00|2025-07-14 14:35:00|
+-----------+-------------+-------------------+-------------------+



In [36]:
from pyspark.sql.functions import max as max_

# Find the most recent sale date in the entire dataset
most_recent_date = sales_df.select(max_("sale_date")).first()[0]
print(f"Most recent sale date: {most_recent_date}")

from pyspark.sql.functions import to_date, lit, datediff

# Calculate the date 30 days before the most recent date
thirty_days_before = sales_df.withColumn("date", to_date(lit(most_recent_date))) \
    .selectExpr("date_sub(date, 30) as cutoff_date").first().cutoff_date

print(f"30-day cutoff date: {thirty_days_before}")

# Find the last purchase date for each customer
customer_last_purchase = sales_df.groupBy("customer_id") \
    .agg(max_("sale_date").alias("last_purchase_date"))

# Find customers whose last purchase was before the 30-day cutoff
churned_customers = customer_last_purchase.filter(col("last_purchase_date") < lit(thirty_days_before))

churned_customers_with_names = churned_customers.join(customer_df, "customer_id") \
    .select("customer_id", "customer_name", "last_purchase_date")

churned_customers_with_names.show()

Most recent sale date: 2025-07-15 16:00:00
30-day cutoff date: 2025-06-15
+-----------+-------------+------------------+
|customer_id|customer_name|last_purchase_date|
+-----------+-------------+------------------+
+-----------+-------------+------------------+

